In [1]:
import copy
import os
import sys
import time
import PyKDL as kdl
import moveit_commander
import numpy as np
import rospy
from geometry_msgs.msg import Twist
from kdl_parser_py.urdf import treeFromParam
from sensor_msgs.msg import JointState
from tf_conversions import transformations

In [2]:
rospy.init_node('ros_test', anonymous=True)

moveit_commander.roscpp_initialize(sys.argv)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)
# This is very significant！
move_group.set_end_effector_link('tool0')
# move_group.get_jacobian_matrix is not affected, so you cannot use it directly
planning_frame = move_group.get_planning_frame()
eef_link = move_group.get_end_effector_link()
group_names = robot.get_group_names()
current_state = robot.get_current_state()

In [3]:
q = np.array(move_group.get_random_joint_values())
dq = np.array(move_group.get_random_joint_values())

In [4]:
_, urdf_tree = treeFromParam("robot_description")
urdf_chain = urdf_tree.getChain('base_link', 'tool0')
fk_solver_vel = kdl.ChainFkSolverVel_recursive(urdf_chain)

def fk_vel(q, dq):
    jq = kdl.JntArrayVel(urdf_chain.getNrOfJoints())
    for i in range(urdf_chain.getNrOfJoints()):
        jq.q[i] = q[i]
        jq.qdot[i] = dq[i]
    framevel = kdl.FrameVel()
    result = fk_solver_vel.JntToCart(jq, framevel)
    if 0 != result:
        raise Exception(f"Error solving TCP velocity: Error code = {result}")
    twist = framevel.GetTwist()
    return twist

fk_vel(q, dq)

[     1.56747,    0.643346,     1.69934,     4.20481,     2.66007,    -5.67392]